In [10]:
import pandas as pd
import numpy as np
import pickle

In [3]:
df = pd.read_csv('test_data.csv')

# feature engineer

In [15]:
# df['have_prev_address'] = np.where(df['prev_address_months_count']<0, 0, 1)

# df['have_initial_bal'] = np.where(df['intended_balcon_amount']<=0, 0, 1)

# df['have_bank_months_count'] = np.where(df['prev_address_months_count']<0, 0, 1)

In [16]:
df['bank_integration'] = df['bank_months_count'] + df['has_other_cards']*df['bank_months_count']

df['income_credit_limit_ratio'] = df['income'] / df['proposed_credit_limit']

# standardisation

In [71]:
cluster_features = [
            'fraud_bool',
            'income',
            'name_email_similarity',
            'current_address_months_count',
            'customer_age',
            'days_since_request',
            'payment_type',
            'zip_count_4w',
            'bank_branch_count_8w',
            'date_of_birth_distinct_emails_4w',
            'employment_status',
            'credit_risk_score',
            'email_is_free',
            'housing_status',
            'phone_home_valid',
            'phone_mobile_valid',
            'bank_months_count', 'have_bank_months_count',
            'has_other_cards',
            'proposed_credit_limit',
            'foreign_request',
            'device_os',
            'device_distinct_emails_8w',
            'bank_integration',
            'income_credit_limit_ratio',
            'have_prev_address',
            'have_initial_bal'
                ]

In [72]:
numerical_features = [
#             'fraud_bool',
            'income',
            'name_email_similarity',
            'current_address_months_count',
            'customer_age',
            'days_since_request',
#             'payment_type',
            'zip_count_4w',
            'bank_branch_count_8w',
            'date_of_birth_distinct_emails_4w',
#             'employment_status',
            'credit_risk_score',
#             'email_is_free',
#             'housing_status',
#             'phone_home_valid',
#             'phone_mobile_valid',
            'bank_months_count', 
#             'has_other_cards',
            'proposed_credit_limit',
#             'foreign_request',
#             'device_os',
            'device_distinct_emails_8w',
            'bank_integration',
            'income_credit_limit_ratio',
#             'have_prev_address',
#             'have_initial_bal'
#             'have_bank_months_count',
                ]

In [73]:
standardisation_dict = pickle.load(open('standardisation_dict.pkl', 'rb'))

In [74]:
df_scaled = df[cluster_features]

for col in numerical_features:
    mean = standardisation_dict['mean'][col] 
    std = standardisation_dict['std'][col]
    df_scaled[col] = (df_scaled[col] - mean) / std 

C:\Users\marcu\AppData\Local\Temp\ipykernel_2580\757233552.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scaled[col] = (df_scaled[col] - mean) / std


# predict kmeans_label

In [75]:
kmeans_model = pickle.load(open('kmeans_model.pkl', 'rb'))

In [76]:
categorical_features = [
#             'fraud_bool',
#             'income',
#             'name_email_similarity',
#             'current_address_months_count',
#             'customer_age',
#             'days_since_request',
            'payment_type',
#             'zip_count_4w',
#             'bank_branch_count_8w',
#             'date_of_birth_distinct_emails_4w',
            'employment_status',
#             'credit_risk_score',
            'email_is_free',
            'housing_status',
            'phone_home_valid',
            'phone_mobile_valid',
#             'bank_months_count', 
            'has_other_cards',
#             'proposed_credit_limit',
            'foreign_request',
            'device_os',
#             'device_distinct_emails_8w',
#             'bank_integration',
#             'income_credit_limit_ratio',
            'have_prev_address',
            'have_initial_bal',
#             'have_bank_months_count'
                ]

In [77]:
df_cluster = pd.get_dummies(df_scaled, columns=categorical_features)
df_cluster.head()

,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,zip_count_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,credit_risk_score,...,foreign_request_1,device_os_linux,device_os_macintosh,device_os_other,device_os_windows,device_os_x11,have_prev_address_0,have_prev_address_1,have_initial_bal_0,have_initial_bal_1
0,1,1.096490,-0.907650,0.052871,0.411913,-0.185651,1.191986,-0.388153,-0.432285,0.547005,...,0,0,0,0,1,0,1,0,1,0
1,1,-0.283780,0.827620,1.147445,-0.390363,-0.184501,0.442970,-0.385933,-0.038417,1.306124,...,0,0,0,0,1,0,1,0,1,0
2,1,-1.664050,-1.172983,2.445135,1.214189,-0.183207,-0.096601,-0.388153,-1.220021,0.235224,...,0,0,1,0,0,0,1,0,1,0
3,1,1.096490,1.431589,1.418267,2.016464,-0.186225,-0.864567,-0.363732,-1.416955,1.075677,...,0,0,0,0,1,0,1,0,1,0
4,1,0.061288,1.465208,-0.172814,-0.390363,-0.185724,1.628829,-0.388153,-0.235351,2.295690,...,0,0,0,1,0,0,1,0,1,0


In [66]:
labels = kmeans_model.predict(df_cluster)

In [78]:
df_scaled['kmeans_label'] = labels
df_scaled.head()

,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,payment_type,zip_count_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,...,has_other_cards,proposed_credit_limit,foreign_request,device_os,device_distinct_emails_8w,bank_integration,income_credit_limit_ratio,have_prev_address,have_initial_bal,kmeans_label
0,1,1.096490,-0.907650,0.052871,0.411913,-0.185651,AC,1.191986,-0.388153,-0.432285,...,1,-0.126792,0,windows,-0.146232,-1.924218,-8.647952,0,0,1
1,1,-0.283780,0.827620,1.147445,-0.390363,-0.184501,AD,0.442970,-0.385933,-0.038417,...,0,2.716336,0,windows,-0.146232,0.753171,-0.104472,0,0,1
2,1,-1.664050,-1.172983,2.445135,1.214189,-0.183207,AC,-0.096601,-0.388153,-1.220021,...,0,2.526794,0,macintosh,-0.146232,-0.962109,-0.658562,0,0,1
3,1,1.096490,1.431589,1.418267,2.016464,-0.186225,AD,-0.864567,-0.363732,-1.416955,...,0,-0.126792,0,windows,-0.146232,-0.798749,-8.647952,0,0,1
4,1,0.061288,1.465208,-0.172814,-0.390363,-0.185724,AC,1.628829,-0.388153,-0.235351,...,0,2.716336,0,other,-0.146232,-0.962109,0.022563,0,0,1


# use labels for new column

In [79]:
# df_cluster = pd.DataFrame({
#     'housing_status': ['BA', 'AA', 'CA', 'BA', 'BA'],
#     'kmeans_label': [0, 1, 0, 1, 1],
#     'fraud_bool': [1, 0, 1, 0, 1],
#     'device_os': ['windows', 'windows', 'android', 'windows', 'android'],
#     'date_of_birth_distinct_emails_4w': [0.5, -1.5, 0.0, 2.0, -2.0],
#     'credit_risk_score': [0.3, 2.5, 0.2, 0.6, 1.7],
#     'income': [0.5, 2.0, 1.5, 3.0, 2.5],
#     'payment_type': ['AC', 'CC', 'DC', 'AC', 'DC'],
#     'bank_branch_count_8w': [-0.5, 0.1, -0.4, 0.2, -0.1],
#     'customer_age': [25, 30, 35, 40, 45],
#     'proposed_credit_limit': [3.0, 2.5, 2.0, 2.6, 3.5],
#     'phone_home_valid': [1, 0, 1, 0, 1],
#     'has_other_cards': [0, 1, 0, 1, 0],
#     'email_is_free': [1, 0, 1, 0, 1],
#     'bank_months_count': [-0.8, -1.0, -0.5, -0.9, -1.1],
#     'kmeans_label': [0, 2, 1, 3, 3]
# })
df_cluster = df_scaled

In [82]:
condition_list = [
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['proposed_credit_limit'] >= 2.474),
    (df_cluster['housing_status']=='BA') & (df_cluster['kmeans_label']==0),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA') & (df_cluster['bank_months_count'] <= -0.901),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA') & (df_cluster['payment_type'] == 'AC'),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA') & (df_cluster['income']>1),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA') & (df_cluster['phone_home_valid'] == 0),
    (df_cluster['kmeans_label']==0) & (df_cluster['credit_risk_score']>=0.4),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA') & (df_cluster['customer_age']>1),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['credit_risk_score'] >= 1.6),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==0),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['date_of_birth_distinct_emails_4w'] <= -1.22),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA') & (df_cluster['bank_branch_count_8w']<= 0.189 ),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA') & (df_cluster['date_of_birth_distinct_emails_4w']<=0.0207),
    (df_cluster['phone_home_valid']==0) & (df_cluster['kmeans_label']==1) & (df_cluster['date_of_birth_distinct_emails_4w'] <= -1.182),
    (df_cluster['has_other_cards']==0) & (df_cluster['kmeans_label']==1) & (df_cluster['date_of_birth_distinct_emails_4w'] <= -1.309),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['bank_months_count'] <= -0.901),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA'),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['income']>1),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['payment_type'] == 'AC'),
    (df_cluster['credit_risk_score']>= 2.2),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['bank_branch_count_8w'] < -0.385),
    (df_cluster['has_other_cards']==0) & (df_cluster['kmeans_label']==1) & (df_cluster['credit_risk_score'] >= 1.8),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['phone_home_valid'] == 0),
    (df_cluster['phone_home_valid']==0) & (df_cluster['kmeans_label']==1) & (df_cluster['housing_status'] == 'BA'),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['email_is_free'] == 1),
    (df_cluster['date_of_birth_distinct_emails_4w'] <= -1.409) & (df_cluster['kmeans_label']==1),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1) & (df_cluster['has_other_cards'] == 0),
    (df_cluster['device_os']=='windows') & (df_cluster['kmeans_label']==1),
    (df_cluster['kmeans_label']==1) & (df_cluster['housing_status']=='BA'),
    (df_cluster['kmeans_label']==0),
    (df_cluster['kmeans_label']==0) & (df_cluster['date_of_birth_distinct_emails_4w'] > 1.4),
    (df_cluster['kmeans_label']==4) & (df_cluster['housing_status']=='BA'),
    (df_cluster['kmeans_label']==3) & (df_cluster['housing_status']=='BA'),
    (df_cluster['kmeans_label']==1),
    (df_cluster['device_os']=='windows'),
    (df_cluster['kmeans_label']==4),
    (df_cluster['kmeans_label']==3),
    (df_cluster['kmeans_label']==2)
]

prob_list = [0.784,
                     0.7182320441988951,
                     0.684322033898305,
                     0.678609062170706,
                     0.6585887384176764,
                     0.6550104384133612,
                     0.642,
                     0.6086036671368125,
                     0.6058897243107769,
                     0.6021505376344086,
                     0.5946445060018467,
                     0.5906344410876133,
                     0.5895710681244744,
                     0.5855072463768116,
                     0.5775193798449613,
                     0.5737704918032787,
                     0.5729963418689724,
                     0.5713582677165354,
                     0.5700615174299385,
                     0.5695159629248198,
                     0.5664575014261266,
                     0.558433734939759,
                     0.5507722642129478,
                     0.532562729066817,
                     0.5224358974358975,
                     0.5131782945736434,
                     0.5012048192771085,
                     0.45865268253327957,
                     0.4510290986515259,
                     0.4230992455020313,
                     0.3706293706293706,
                     0.3555787278415016,
                     0.3225643614336194,
                     0.3137490747594375,
                     0.30641330166270786,
                     0.13373963094633487,
                     0.1131566028473245,
                     0.06378108417438352]

In [83]:
df_cluster['kmeans_prob'] = np.select(condition_list, prob_list, 0)

In [84]:
df_cluster

,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,payment_type,zip_count_4w,bank_branch_count_8w,date_of_birth_distinct_emails_4w,...,proposed_credit_limit,foreign_request,device_os,device_distinct_emails_8w,bank_integration,income_credit_limit_ratio,have_prev_address,have_initial_bal,kmeans_label,kmeans_prob
0,1,1.096490,-0.907650,0.052871,0.411913,-0.185651,AC,1.191986,-0.388153,-0.432285,...,-0.126792,0,windows,-0.146232,-1.924218,-8.647952,0,0,1,0.684322
1,1,-0.283780,0.827620,1.147445,-0.390363,-0.184501,AD,0.442970,-0.385933,-0.038417,...,2.716336,0,windows,-0.146232,0.753171,-0.104472,0,0,1,0.784000
2,1,-1.664050,-1.172983,2.445135,1.214189,-0.183207,AC,-0.096601,-0.388153,-1.220021,...,2.526794,0,macintosh,-0.146232,-0.962109,-0.658562,0,0,1,0.585507
3,1,1.096490,1.431589,1.418267,2.016464,-0.186225,AD,-0.864567,-0.363732,-1.416955,...,-0.126792,0,windows,-0.146232,-0.798749,-8.647952,0,0,1,0.658589
4,1,0.061288,1.465208,-0.172814,-0.390363,-0.185724,AC,1.628829,-0.388153,-0.235351,...,2.716336,0,other,-0.146232,-0.962109,0.022563,0,0,1,0.569516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19787,0,-0.628847,1.048779,-0.556479,2.016464,-0.182222,AB,-0.305049,-0.368173,-1.023087,...,-0.126792,0,linux,-0.146232,1.569971,4.959680,0,0,3,0.322564
19788,0,0.751423,-1.417962,-0.928859,-0.390363,2.973405,AC,0.004132,-0.388153,-1.023087,...,-0.695417,0,linux,-0.146232,-0.962109,-1.080535,0,0,4,0.133740
19789,0,-1.664050,1.007276,-0.184098,-1.994915,-0.186493,AA,1.172039,1.558825,-0.235351,...,0.820917,0,macintosh,-0.146232,1.651651,-2.027062,0,1,3,0.113157
19790,0,0.061288,1.056754,0.052871,-1.192639,-0.181318,AA,-0.143477,-0.381493,-0.826153,...,-0.695417,0,other,-0.146232,0.753171,-0.088131,0,1,3,0.113157
